In [2]:
import open3d as o3d
print(o3d.__version__)
import numpy as np
import pandas as pd
import polars as pl
import colorsys

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
0.18.0


In [48]:
print("kota_circuit")
ply_point_cloud = o3d.data.PLYPointCloud()
pcd = o3d.io.read_point_cloud("../Open3D-01/Dataset/kota_circuit2.ply")
print(pcd)
print(np.asarray(pcd.points))

kota_circuit
PointCloud with 14640197 points.
[[  82.07930756 -147.18885803  -78.15903473]
 [  97.60070801  149.98822021  -53.49351883]
 [  70.49710846 -229.24028015  -78.54177856]
 ...
 [  26.64775658  -36.49408722  -89.59648132]
 [ -88.472229     49.91682816  -75.39753723]
 [ -23.17474937  335.25888062 -102.58750916]]


In [49]:
print("render file")
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

render file


In [50]:
# for cropping area by manual -> cropped_kota_circuit
# o3d.visualization.draw_geometries_with_editing([pcd])
crop_pcd = o3d.io.read_point_cloud("Data/cropped_road.ply")
o3d.visualization.draw_geometries([crop_pcd])

In [51]:
print(crop_pcd)
print(np.asarray(crop_pcd.points))

PointCloud with 2915323 points.
[[-18.5238533  105.0320282  -95.51685333]
 [ 27.83203697  29.22127914 -90.54051971]
 [ 57.42870331 -15.37507629 -89.91012573]
 ...
 [-24.05081367 228.32498169 -94.57554626]
 [ 39.06476593 -49.5741539  -89.21252441]
 [ 26.64775658 -36.49408722 -89.59648132]]


In [52]:
print("Normal Vector")
crop_pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([crop_pcd],
                                  point_show_normal=True)

Normal Vector


In [53]:
# Setting
point_cloud = np.asarray(crop_pcd.points) 
color_point = np.asarray(crop_pcd.colors)
normal_vector = np.asarray(crop_pcd.normals)

print("point cloud: \n", point_cloud)
print()
print("normal vector: \n", normal_vector)
print()
print("color point: \n", color_point)

point cloud: 
 [[-18.5238533  105.0320282  -95.51685333]
 [ 27.83203697  29.22127914 -90.54051971]
 [ 57.42870331 -15.37507629 -89.91012573]
 ...
 [-24.05081367 228.32498169 -94.57554626]
 [ 39.06476593 -49.5741539  -89.21252441]
 [ 26.64775658 -36.49408722 -89.59648132]]

normal vector: 
 [[ 0.          0.          1.        ]
 [ 0.          0.          1.        ]
 [ 0.87776204  0.30015999  0.37341369]
 ...
 [ 0.09973728  0.12625676 -0.98697097]
 [-0.70574554  0.70522761 -0.06765539]
 [ 0.0470352  -0.9870053   0.15364969]]

color point: 
 [[0.19215686 0.17254902 0.16078431]
 [0.4        0.43921569 0.26666667]
 [0.55294118 0.55294118 0.57254902]
 ...
 [0.74509804 0.72941176 0.6627451 ]
 [0.41960784 0.44705882 0.29019608]
 [0.35686275 0.32941176 0.23137255]]


In [54]:
# for filter by color
for i in range(len(point_cloud)):
    r = color_point[i][0]
    g = color_point[i][1]
    b = color_point[i][2]
    h, s, v = colorsys.rgb_to_hsv(r,g,b)
    
    # check if it isn't green or black
    cond_h = not (h >= 80/360*100 and h <= 260/360*100)
    cond_s = (s >= 0 and s <= 20/100)
    cond_v = v>50/100
    
    if (cond_h and cond_s and cond_v):           # this isn't green condition
        normal_vector[i][2] = normal_vector[i][2]
    else:                                        # this is green condition
        normal_vector[i][2] = 0.5

In [55]:
# Save to .ply in ASCII
output_file_path = "Data/cropped_ASCII.ply"

o3d.io.write_point_cloud(output_file_path, crop_pcd, write_ascii=True)

True

In [56]:
# set to Dataframe
file_path = 'Data/cropped_ASCII.ply'

data = []
check = False

with open(file_path, 'r') as file:
    for line in file:
        if check :
            row = line.strip().split()
            data.append(row)
        elif "end_header" in line:
            check = True
            
df = pl.DataFrame(data, schema=['x','y','z','nx','ny','nz','r','g','b'])
print(df)

shape: (2_915_323, 9)
┌──────────┬──────────┬──────────┬───────────┬───┬───────────┬─────┬─────┬─────┐
│ x        ┆ y        ┆ z        ┆ nx        ┆ … ┆ nz        ┆ r   ┆ g   ┆ b   │
│ ---      ┆ ---      ┆ ---      ┆ ---       ┆   ┆ ---       ┆ --- ┆ --- ┆ --- │
│ str      ┆ str      ┆ str      ┆ str       ┆   ┆ str       ┆ str ┆ str ┆ str │
╞══════════╪══════════╪══════════╪═══════════╪═══╪═══════════╪═════╪═════╪═════╡
│ -18.5239 ┆ 105.032  ┆ -95.5169 ┆ 0         ┆ … ┆ 0.5       ┆ 49  ┆ 44  ┆ 41  │
│ 27.832   ┆ 29.2213  ┆ -90.5405 ┆ 0         ┆ … ┆ 0.5       ┆ 102 ┆ 112 ┆ 68  │
│ 57.4287  ┆ -15.3751 ┆ -89.9101 ┆ 0.877762  ┆ … ┆ 0.373414  ┆ 141 ┆ 141 ┆ 146 │
│ -38.122  ┆ 161.518  ┆ -93.5807 ┆ 0         ┆ … ┆ 1         ┆ 189 ┆ 197 ┆ 201 │
│ -54.9689 ┆ 99.5871  ┆ -92.1375 ┆ 0.193967  ┆ … ┆ -0.875274 ┆ 213 ┆ 219 ┆ 219 │
│ …        ┆ …        ┆ …        ┆ …         ┆ … ┆ …         ┆ …   ┆ …   ┆ …   │
│ 37.2285  ┆ 27.1855  ┆ -90.6008 ┆ 0.0162607 ┆ … ┆ 0.5       ┆ 141 ┆ 142 ┆ 105 │
│ 32.0

In [57]:
filter_crop = df.filter(
    # with normal vector
    (df['nz'] >= 0.9) &
    (-0.2<df['ny']) & (df['ny']<0.2) &
    (-0.2<df['nx']) & (df['nx']<0.2)
    # with color
    # ~( (df['g']>=df['b']) & (df['g']>=df['r']) ) 
)

print(filter_crop)

shape: (844_085, 9)
┌──────────┬─────────┬──────────┬─────┬───┬─────┬─────┬─────┬─────┐
│ x        ┆ y       ┆ z        ┆ nx  ┆ … ┆ nz  ┆ r   ┆ g   ┆ b   │
│ ---      ┆ ---     ┆ ---      ┆ --- ┆   ┆ --- ┆ --- ┆ --- ┆ --- │
│ str      ┆ str     ┆ str      ┆ str ┆   ┆ str ┆ str ┆ str ┆ str │
╞══════════╪═════════╪══════════╪═════╪═══╪═════╪═════╪═════╪═════╡
│ -38.122  ┆ 161.518 ┆ -93.5807 ┆ 0   ┆ … ┆ 1   ┆ 189 ┆ 197 ┆ 201 │
│ -29.8598 ┆ 256.955 ┆ -94.7837 ┆ 0   ┆ … ┆ 1   ┆ 188 ┆ 187 ┆ 156 │
│ -24.2454 ┆ 221.546 ┆ -94.5748 ┆ 0   ┆ … ┆ 1   ┆ 163 ┆ 157 ┆ 147 │
│ 12.9111  ┆ 88.0813 ┆ -91.3888 ┆ 0   ┆ … ┆ 1   ┆ 160 ┆ 161 ┆ 166 │
│ 37.1636  ┆ 71.8371 ┆ -91.612  ┆ 0   ┆ … ┆ 1   ┆ 163 ┆ 162 ┆ 166 │
│ …        ┆ …       ┆ …        ┆ …   ┆ … ┆ …   ┆ …   ┆ …   ┆ …   │
│ -14.6972 ┆ 263.148 ┆ -94.6149 ┆ 0   ┆ … ┆ 1   ┆ 181 ┆ 185 ┆ 181 │
│ 65.9688  ┆ -24.609 ┆ -89.5823 ┆ 0   ┆ … ┆ 1   ┆ 133 ┆ 138 ┆ 141 │
│ 31.743   ┆ 147.465 ┆ -93.2448 ┆ 0   ┆ … ┆ 1   ┆ 201 ┆ 211 ┆ 213 │
│ 18.1897  ┆ 88.2536 ┆ -91.5

In [58]:
# Credit by Kit

with open("Data/filtered_crop.ply", 'w') as file:
    with open("Data/cropped_ASCII.ply", 'r') as ori_file:
        for line in ori_file:
            if "end_header" in line:
                file.write(line)
                ori_file.close()
                break
            else:
                if "element vertex" in line:
                    file.write("element vertex {}\n".format(filter_crop.shape[0]))
                    continue
                file.write(line)
    for row in filter_crop.to_dicts():
        file.write(str(row['x']) + ' ')
        file.write(str(row['y']) + ' ')
        file.write(str(row['z']) + ' ')
        file.write(str(row['nx']) + ' ')
        file.write(str(row['ny']) + ' ')
        file.write(str(row['nz']) + ' ')
        file.write(str(row['r']) + ' ')
        file.write(str(row['g']) + ' ')
        file.write(str(row['b']) + '\n')

In [59]:
path_filter_crop = "Data/filtered_crop.ply"

filter_crop_pcd = o3d.io.read_point_cloud(path_filter_crop)
print(filter_crop_pcd)
o3d.visualization.draw_geometries([filter_crop_pcd])

PointCloud with 844085 points.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
